In [16]:
import pyodbc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
dados_conexao = (
    "Driver={SQL Server};"
    "Server=DESKTOP-2K9QLKU\SQLSERVER;"
    "Database=AdventureWorksDW2022;"
)
conexao = pyodbc.connect(dados_conexao)
print("Conexao bem sucedida")

Conexao bem sucedida


<>:9: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\S'
C:\Users\ANTONIO NETO\AppData\Local\Temp\ipykernel_58272\3343617593.py:9: SyntaxWarning: invalid escape sequence '\S'
  "Server=DESKTOP-2K9QLKU\SQLSERVER;"


In [17]:
codigo_sql = """
SELECT 
    dst.SalesTerritoryRegion AS Regiao,
    CONCAT(dsd.CalendarYear, '-', FORMAT(dsd.MonthNumberOfYear, '00')) AS AnoMes, 
    dpc.EnglishProductCategoryName AS CategoriaProduto,  
    SUM(fis.OrderQuantity) AS QuantidadeVendida,  
    ROUND(SUM(fis.SalesAmount), 2) AS TotalVendas
FROM 
    dbo.FactInternetSales fis
JOIN 
    dbo.DimSalesTerritory dst ON fis.SalesTerritoryKey = dst.SalesTerritoryKey
JOIN 
    dbo.DimDate dsd ON fis.OrderDateKey = dsd.DateKey
JOIN 
    dbo.DimProduct dp ON fis.ProductKey = dp.ProductKey
JOIN 
    dbo.DimProductSubcategory dps ON dp.ProductSubcategoryKey = dps.ProductSubcategoryKey
JOIN 
    dbo.DimProductCategory dpc ON dps.ProductCategoryKey = dpc.ProductCategoryKey
GROUP BY 
    dst.SalesTerritoryRegion,
    dsd.CalendarYear,
    dsd.MonthNumberOfYear,
    dpc.EnglishProductCategoryName
ORDER BY 
    dst.SalesTerritoryRegion,
    dsd.CalendarYear, 
    dsd.MonthNumberOfYear, 
    dpc.EnglishProductCategoryName;
"""

dados = pd.read_sql(codigo_sql, conexao)
conexao.close()

C:\Users\ANTONIO NETO\AppData\Local\Temp\ipykernel_58272\1724192478.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dados = pd.read_sql(codigo_sql, conexao)


In [18]:
print(dados.head())

      Regiao   AnoMes CategoriaProduto  QuantidadeVendida  TotalVendas
0  Australia  2010-12            Bikes                  6     20909.78
1  Australia  2011-01            Bikes                 62    207130.29
2  Australia  2011-02            Bikes                 62    204151.12
3  Australia  2011-03            Bikes                 53    177393.50
4  Australia  2011-04            Bikes                 64    214593.39


In [19]:
categorias_unicas = dados['CategoriaProduto'].unique()

def plotar_mapa_de_calor(categoria_filtro):
 
    dados_filtrados = dados[dados['CategoriaProduto'] == categoria_filtro]
    if dados_filtrados.empty:
        print("Não há dados suficientes para esta categoria.")
        return
    dados_pivot = dados_filtrados.pivot_table(
        index='Regiao',   
        columns='AnoMes',   
        values='TotalVendas',
        aggfunc='sum',
        fill_value=0
    )
    plt.figure(figsize=(14, 8))

    sns.heatmap(
        dados_pivot,
        cmap='Blues',  
        cbar_kws={'label': 'Total de Vendas'} 
    )
    plt.title(f'Mapa de Calor das Vendas por Região e Mês - Categoria: {categoria_filtro}')
    plt.xlabel('Ano-Mês')
    plt.ylabel('Região')

    plt.tight_layout()
    plt.show()
categoria_dropdown = widgets.Dropdown(
    options=categorias_unicas,
    value=categorias_unicas[0],
    description='Categoria:',
)
interact(plotar_mapa_de_calor, categoria_filtro=categoria_dropdown)

interactive(children=(Dropdown(description='Categoria:', options=('Bikes', 'Accessories', 'Clothing'), value='…

<function __main__.plotar_mapa_de_calor(categoria_filtro)>